In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_winare_dsd_golive 
AS
WITH bu_mnf_golive AS (
    SELECT *
    FROM (
    SELECT explode(array('1500', '2000')) AS business_unit
) bu
CROSS JOIN (
    SELECT explode(array(
        '1006', '1003', '1061',
        '1008', '1016', '1022', '1023', '1025', '1026', '1049', '1059', '1110', '1199', '1219',
        '1004', '1017', '1009', '1040', '1010', '2291', '1091', '1043', '1055', '1177'
    )) AS manufacturer_id
) mnf
),
main AS (
    SELECT DISTINCT
        a.store_id,
        a.product_id,
        'YES' golive
    FROM
        {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dsd a
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b ON a.store_id = b.store_id
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product c ON a.product_id = c.product_id
        INNER JOIN bu_mnf_golive d ON a.business_unit = d.business_unit AND a.manufacturer_id = d.manufacturer_id
    WHERE
        b.store_status_desc = "Đang hoạt động"
        AND a.business_unit IN ("1500", "2000")
        AND b.is_win_plus <> 'YES'
        AND c.temperature = 'A3'
)
SELECT * FROM main;
""")